In [ ]:
import pandas as pd
import numpy as np
from matplotlib_venn import venn3
from matplotlib_venn import venn2
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from cycler import cycler
%matplotlib inline

## First load the HES datasets

In [ ]:
location = 'C:/Users/Andrew Devereau/Documents/GeL/secondary data/Data applications/HES return October 2016/'
filename = 'NIC12784_AE.txt'

In [ ]:
dataAE = pd.read_csv(location+filename, sep='|')  #this is the A&E data set

In [ ]:
filename = 'NIC12784_CC.txt'  #Critical care dataset
dataCC = pd.read_csv(location+filename, sep='|')

In [ ]:
filename = 'NIC12784_APC.txt'   #this is the admitted patient care dataset
dataAPC = pd.read_csv(location+filename, sep='|')

In [ ]:
filename = 'NIC12784_OP.txt'   #this is the outpatient dataset
dataOP = pd.read_csv(location+filename, sep='|')

### Get the cancer participants

In [ ]:
filename = 'Cancer.csv'
cancerIDs = pd.read_csv(location+filename, sep=',')  #get the set of cancer patients sent to HES

In [ ]:
cancerIDs = cancerIDs.rename(columns = {'Participant Id':'STUDY_ID'})  #rename the participant Id to STUDY_ID to match results

### Load the ICD, AE and specialty code lookup files and define lookup functions

In [ ]:
filename = 'Tabular.xml'       #get ICD10 code list to convert codes to names
ICDtree = ET.parse(location+filename)
ICDroot = ICDtree.getroot()

In [ ]:
def getICD(search):  #look up ICD disease name from a search code
    for code in ICDroot.iter('diag'):
        name = code.find('name').text
        if name == search:
            desc = code.find('desc').text
            return(search + ' ' + desc)

In [ ]:
filename = 'ae.txt'
aecode = {}
with open(location+filename) as f:
    for line in f:
       (key, val) = line.split('\t')
       aecode[int(key)] = val.strip()

In [ ]:
def getAE(search): #look up diag code in AE data 
    try:
        return (aecode[int(search)])
    except:
        return('None')

In [ ]:
filename = 'spefCode.txt'
spefCode = {}
with open(location+filename) as f:
    for line in f:
       (key, val) = line.split('\t')
       spefCode[key] = val.strip()

In [ ]:
def getSpef(search): #look up speciality codes
    try:
        return (spefCode[search])
    except:
        return('None')

In [ ]:
filename = 'OPCS47 CodesAndTitles Nov 2013 V1.0.txt'
OPCSCode = {}
with open(location+filename) as f:
    for line in f:
        (key, val) = line.split('\t')
        OPCSCode[key] = val.strip()

In [ ]:
def getOPCS (code): #get the OPCS description from the code
    if len(code) > 3: #HES gives OPCS codes as a letter with three numbers with no decimal point - add one if the code is four characters long
        code = code[:3] + '.' + code[3:]
    try:
        return (OPCSCode[code])
    except:
        return('None')

In [ ]:
def getNo (prefix, min, max): #generate a serial number string up to max with prefix, e.g. 'OPERTN_01..24'
    for no in range (min ,max+1):
        if no < 10:
            yield (prefix + '0' + str(no))
        else:
            yield (prefix + str(no))

### merge CC with APC to match study id with susrecid

In [ ]:
CCmerge = pd.merge(dataCC, dataAPC, on='SUSRECID', how = 'left')   

### Make cancer subsets of the HES data sets

In [ ]:
cancerCC = CCmerge[(CCmerge['STUDY_ID'].isin(cancerIDs['STUDY_ID']))] 

In [ ]:
cancerCC.info(verbose=True, null_counts=True)

In [ ]:
cancerAE = dataAE[(dataAE['STUDY_ID'].isin(cancerIDs['STUDY_ID']))] 

In [ ]:
cancerAE.info()

In [ ]:
len(cancerAE['STUDY_ID'].value_counts())

In [ ]:
cancerAE.info(verbose=True, null_counts=True)

In [ ]:
cancerAPC = dataAPC[(dataAPC['STUDY_ID'].isin(cancerIDs['STUDY_ID']))]   #get cancer patients from APC results

In [ ]:
cancerAPC.info()

In [ ]:
len(cancerAPC['STUDY_ID'].value_counts())

In [ ]:
cancerAPC.info(verbose=True, null_counts=True)

In [ ]:
cancerOP = dataOP[(dataOP['STUDY_ID'].isin(cancerIDs['STUDY_ID']))]   #get cancer patients from OP results

In [ ]:
cancerOP.info(verbose=True, null_counts=True)

### Add consent dates to the cancer data sets

In [ ]:
filename = 'cancer_consent_2016-11-28_16-34-24.xlsx'
consentDates = pd.read_excel(location+filename)  #get consent dates

In [ ]:
consentDates = consentDates.drop_duplicates('Participant Identifiers Id', keep='first')  #remove duplicate records

In [ ]:
consentDates.drop(['Metadata Date', 'Event Date', 'Consent Given Id'], axis=1, inplace=True) #remove all columns except participant ID and date

In [ ]:
consentDates.rename(columns={'Participant Identifiers Id': 'STUDY_ID'}, inplace=True) #rename participant to STUDY_ID

In [ ]:
consentDates.head()

In [ ]:
cancerIDs[~(cancerIDs['STUDY_ID'].isin(consentDates['STUDY_ID']))]  #check for any cancerIDs not in the consentDates set

In [ ]:
cancerCC = pd.merge(cancerCC, consentDates, on='STUDY_ID', how = 'left')

In [ ]:
cancerAE = pd.merge(cancerAE, consentDates, on='STUDY_ID', how = 'left')
cancerAPC = pd.merge(cancerAPC, consentDates, on='STUDY_ID', how = 'left')
cancerOP = pd.merge(cancerOP, consentDates, on='STUDY_ID', how = 'left')

### Look for cancer therapy history using OPCS codes in APC and OP

In [ ]:
#first aggregate all operation codes to look for errors, frequency etc.
nameGen = getNo('OPERTN_', 24)   #get a generator to give the OPERTN_ column headings
countList = []  #a list of series each holding value counts for a column of opertn codes
for n in range(24): 
    countList.append(cancerAPC.groupby(next(nameGen)).size())  #get a series for each column with term frequencies
counts = pd.concat(countList)   #concatenate all counts into one series

In [ ]:
len(counts.groupby(counts.index).sum())  #there are 1766 different operation codes used for cancer participants

There are no cases of opertn values mixed between ints, strings and floats. There are many codes not given ('-') plus a few '&'

In [ ]:
opcsFreq = counts.groupby(counts.index).sum().sort_values(ascending=False).to_frame()
opcsFreq.rename(columns={0:'frequency'}, inplace=True)
opcsFreq['Operation'] = opcsFreq.index.map(getOPCS)

In [ ]:
opcsFreq

In [ ]:
opcsFreq.to_excel('operations.xlsx')  #get excel file to check through all codes for malformed values

In [ ]:
len(countList[0].groupby(countList[0].index).sum()) #just get the primary OPCS code i.e. OPERTN_01

Get the primary operation code frequency

In [ ]:
opcsPrimaryFreq = countList[0].groupby(countList[0].index).sum().sort_values(ascending=False).to_frame()
opcsPrimaryFreq.rename(columns={0:'frequency'}, inplace=True)
opcsPrimaryFreq['Operation'] = opcsPrimaryFreq.index.map(getOPCS)

In [ ]:
opcsPrimaryFreq

Get the secondary (OPERTN_02 - 24) operation frequency

In [ ]:
countsSecond = pd.concat(countList[1:24])   #concatenate all counts into one series for secondary data i.e. OPERTN_02 onwards
opcsSecondFreq = countsSecond.groupby(countsSecond.index).sum().sort_values(ascending=False).to_frame()
opcsSecondFreq.rename(columns={0:'frequency'}, inplace=True)
opcsSecondFreq['Operation'] = opcsSecondFreq.index.map(getOPCS)

In [ ]:
opcsSecondFreq

Get patients with radiotherapy or chemotherapy codes

In [ ]:
X65 = ['X651','X652','X653', 'X654', 'X655', 'X656','X657', 'X658', 'X659'] #radiotherapy delivery
X67 = ['X671', 'Y672', 'Y673', 'Y674', 'Y675', 'Y676', 'Y677', 'Y678', 'Y679'] #Preparation for external beam radiotherapy
X68 = ['X681', 'X682', 'X683', 'X688', 'X689'] #preparation for brachytheraphy
Y90 = ['Y902'] #Radiotherapy NEC
Y91 = ['Y911', 'Y911','Y912','Y913', 'Y914', 'Y915', 'Y918', 'Y919'] #External beam radiotherapy
Y92 = ['Y921', 'Y928', 'Y929'] #Support for preparation for radiotherapy
X3 = ['X352', 'X373', 'X384'] #Intravenous chemotherapy, Intramuscular chemotherapy, Subcutaneous chemotherapy
Y3 = ['Y354', 'Y364'] #Introduction of radioactive substance into organ for brachytherapy NOC, Introduction of non-removable radioactive substance into organ for brachytherapy NOC
Y89 = ['Y891', 'Y892', 'Y898', 'Y899'] #Brachytherapy
X72 = ['X721','X722','X723','X724','X728','X729'] #Delivery of chemotherapy for neoplasm
X70 = ['X721','X702','X703','X704','X705','X708','X709'] #Procurement of drugs for chemotherapy for neoplasm in Bands 1-5
X71 = ['X721','X712','X713','X714','X715','X718','X719'] #Procurement of drugs for chemotherapy for neoplasm in Bands 6-10
X73 = ['X721','X738','X739'] #Delivery of oral chemotherapy for neoplasm
X74 = ['X741', 'X742', 'X748', 'X749'] #Other chemotherapy drugs

In [ ]:
caList = X65+X67+X68+Y90+Y91+Y92+X3+Y3+Y89+X72+X70+X71+X73+X74  #list of OPCS4 codes to indicate cancer treatment

Get patients from APC data with cancer treatment code in any OPERTN field

In [ ]:
APCSet = set()
for field in getNo ('OPERTN_', 24):
    oncSet = set(cancerAPC[(cancerAPC[field].isin(caList)) & (pd.to_datetime(cancerAPC.ADMIDATE) < cancerAPC['Date Of Consent'])]['STUDY_ID'])
    APCSet = APCSet.union(oncSet)
print(len(APCSet))
oncologyAPCPatientsOPCS = cancerAPC[cancerAPC.STUDY_ID.isin(APCSet)]

In [ ]:
len(oncologyAPCPatientsOPCS)

Get patients from OP data with cancer treatment in any OPERTN field

In [ ]:
OPSet = set()
for field in getNo ('OPERTN_',1, 24):
    oncSet = set(cancerOP[(cancerOP[field].isin(caList)) & (pd.to_datetime(cancerOP.APPTDATE) < cancerOP['Date Of Consent'])]['STUDY_ID'])
    OPSet = OPSet.union(oncSet)
print(len(OPSet))
oncologyOPPatientsOPCS = cancerOP[cancerOP.STUDY_ID.isin(OPSet)]

In [ ]:
len(oncologyOPPatientsOPCS)

In [ ]:
fullCaSet = OPSet.union(APCSet)
len(fullCaSet)

In [ ]:
for f in sorted(fullCaSet):
    print (f)

Make subsets of the APC and OP data for the Ca patients. These include all columns.

In [ ]:
cancerHistoryAPC = cancerAPC[(cancerAPC['STUDY_ID'].isin(fullCaSet))]
cancerHistoryOP = cancerOP[(cancerOP['STUDY_ID'].isin(fullCaSet))]

Export spreadsheets of the particpants in the subsets

In [ ]:
paList = list(cancerHistoryAPC['STUDY_ID'])

In [ ]:
len(paList)

In [ ]:
cancerHistoryAPC['STUDY_ID'].value_counts()

In [ ]:
for n in range(0,5):
    
    writer = pd.ExcelWriter(str(paList[n]) + '.xlsx')
    cancerHistoryAPC[(cancerHistoryAPC['STUDY_ID'] == paList[n])].to_excel(writer, sheet_name = 'APC',index=False)
    cancerHistoryOP[(cancerHistoryOP['STUDY_ID'] == paList[n])].to_excel(writer, sheet_name = 'OP',index=False)
    cancerAE[(cancerAE['STUDY_ID'] == paList[n])].to_excel(writer, sheet_name = 'AE',index=False)
    cancerCC[(cancerCC['STUDY_ID'] == paList[n])].to_excel(writer, sheet_name = 'CC',index=False)
    writer.save()

In [ ]:
frameList = []
for field in getNo ('OPERTN_',1, 24):
    subframe = cancerHistoryAPC[['STUDY_ID', field, 'ADMIDATE']]
    frameList.append(subframe)
secondaryAPC = pd.concat(frameList)

In [ ]:
secondaryAPC

In [ ]:
cancerHistoryAPC[(cancerHistoryAPC['STUDY_ID'] == 221000052)][cancerHistoryAPC]

In [ ]:
cancerHistoryAPC = cancerHistoryAPC.rename(columns = {'ADMIDATE':'DATE'}) #rename the date to a common heading of DATE
cancerHistoryOP = cancerHistoryOP.rename(columns = {'APPTDATE':'DATE'}) 

In [ ]:
def getNewID (ID, IDDict):   #function to create new participant ID for plotting purposes
    return IDDict.get(ID)

In [ ]:
IDs = list(np.sort(cancerHistoryAPC['STUDY_ID'].unique()))
newIDs = list(int(x) for x in range(1,len(IDs)))
zipped = zip(IDs, newIDs)
IDDictAPC = dict(zip(IDs, newIDs))   #create a dictionary mapping STUDY_ID to a new smaller ID for plotting of data

In [ ]:
IDs = list(np.sort(cancerHistoryOP['STUDY_ID'].unique()))
newIDs = list(int(x) for x in range(1,len(IDs)))
zipped = zip(IDs, newIDs)
IDDictOP = dict(zip(IDs, newIDs))   #create a dictionary mapping STUDY_ID to a new smaller ID for plotting of data

In [ ]:
cancerHistoryAPC['ID'] = cancerHistoryAPC['STUDY_ID'].apply(getNewID, args =(IDDictAPC,))  #add new column with new ID

In [ ]:
cancerHistoryOP['ID'] = cancerHistoryOP['STUDY_ID'].apply(getNewID, args =(IDDictOP,))  #add new column with new ID

In [ ]:
groups1 = cancerHistoryAPC.groupby('OPERTN_01')   #create a timeseries plot grouped by primary operation code
groups2 = cancerHistoryOP.groupby('OPERTN_01')
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
ax.set_ylabel('Patient number')
#ax.set_xlim(pd.to_datetime('2015'),pd.to_datetime('2016'))
for name, group in groups1:
    if name in caList:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='o', color = 'r', alpha=1)
    else:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='.', color = 'b', alpha=0.5)
for name, group in groups2:
    if name in caList:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='o', color = 'g', alpha=1)
    else:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='.', color = 'b', alpha=0.5)
#ax.legend(loc='best')
ax.title
plt.show()

In [ ]:
frameList = []
for field in getNo ('OPERTN_',2, 24):
    subframe = cancerHistoryAPC[[field, 'DATE', 'ID']]
    subframe.rename(columns={field:'OPERTN'}, inplace=True)
    frameList.append(subframe)
secondaryAPC = pd.concat(frameList)

In [ ]:
frameList = []
for field in getNo ('OPERTN_',2, 24):
    subframe = cancerHistoryOP[[field, 'DATE', 'ID']]
    subframe.rename(columns={field:'OPERTN'}, inplace=True)
    frameList.append(subframe)
secondaryOP = pd.concat(frameList)

In [ ]:
groups1 = secondaryAPC.groupby('OPERTN')   #create a timeseries plot grouped by primary operation code
groups2 = secondaryOP.groupby('OPERTN')
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
ax.set_ylabel('Patient number')
#ax.set_xlim(pd.to_datetime('2015'),pd.to_datetime('2016'))
for name, group in groups1:
    if name in caList:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='o', color = 'r', alpha=1)
    else:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='.', color = 'b', alpha=0.5)
for name, group in groups2:
    if name in caList:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='o', color = 'g', alpha=1)
    else:
        ax.plot(pd.to_datetime(group['DATE']), group['ID'], label = name, linestyle='none', marker='.', color = 'b', alpha=0.5)
#ax.legend(loc='best')
ax.title
plt.show()

Look at intersection of participants with cancer treatment and those with oncology specialty

In [ ]:
oncHES = pd.read_excel('oncologyHES.xlsx')

In [ ]:
oncSet = set(oncHES.iloc[:,0])

In [ ]:
len(oncSet)

In [ ]:
venn2([oncSet, fullCaSet])

In [ ]:
noTreatmentset = fullCaSet- oncSet

In [ ]:
noTreatmentset  #these are people with no oncology history but with a treatment history